## Import  Section

In [1]:
import pandas as pd
import scipy, scipy.stats
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
sns.set(style="white", color_codes=True)
from sklearn import metrics
import numpy as np
import datetime
from calendar import month
import warnings
warnings.filterwarnings("ignore")


## Data Read in 

In [2]:
a_data = pd.read_csv('C:/Users/bswil/Desktop/Team Phoenix GitHub/Data/accident.csv')
a_data.head()

,STATE,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,...,NOT_MIN,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR
0,1,10001,1,1,0,0,0,1,1,127,...,99,2,58,88,88,0,0,0,1,1
1,1,10002,1,1,0,0,0,1,1,83,...,99,22,20,88,88,0,0,0,1,0
2,1,10003,1,1,0,0,0,2,2,11,...,99,1,45,99,99,0,0,0,1,1
3,1,10004,1,1,0,0,0,1,1,45,...,99,1,15,88,88,0,0,0,1,1
4,1,10005,2,2,0,0,0,2,2,45,...,99,7,16,88,88,0,0,0,1,0


# Data Wrangling
## Keep the collumns for analysis 
### look at the data 

In [3]:
a_data1 = a_data[[ 'MONTH', 'FATALS']]
a_data1.head()

,MONTH,FATALS
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [4]:
a_data1.describe()

,MONTH,FATALS
count,32166.000000,32166.000000
mean,6.790680,1.090966
std,3.354665,0.365794
min,1.000000,1.000000
25%,4.000000,1.000000
50%,7.000000,1.000000
75%,10.000000,1.000000
max,12.000000,10.000000


## data wranging 


In [5]:

months  =   {1: 'January',
            2: 'February',
            3: 'March', 
            4: 'April',
            5: 'May', 
            6: 'June', 
            7: 'July',
            8: 'August',
            9: 'September',
            10: 'October', 
            11: 'November', 
            12: 'December'} 

a_data1['MonthR']=a_data1['MONTH'].apply(lambda x: months[x])
 

In [6]:
def SEASONS (series):
    if series == 'December':
        return "Winter"
    elif series == 'February':
        return "Winter"
    elif series == 'January':
        return "Winter"
    elif series == 'September':
        return "Fall"
    elif series == 'October':
        return "Fall"
    elif series == 'November':
        return "Fall"
    
a_data1['SEASON'] = a_data1['MonthR'].apply(SEASONS)
a_data1['SEASON'].value_counts(sort=False)

Winter    7117
Fall      8608
Name: SEASON, dtype: int64

In [7]:
a_data1.head()

,MONTH,FATALS,MonthR,SEASON
0,1,1,January,Winter
1,1,1,January,Winter
2,1,1,January,Winter
3,1,1,January,Winter
4,1,1,January,Winter


In [8]:
a_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32166 entries, 0 to 32165
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   MONTH   32166 non-null  int64 
 1   FATALS  32166 non-null  int64 
 2   MonthR  32166 non-null  object
 3   SEASON  15725 non-null  object
dtypes: int64(2), object(2)
memory usage: 1005.3+ KB


In [9]:
#get unique values in Season column
a_data1.SEASON.unique()

array(['Winter', None, 'Fall'], dtype=object)

In [10]:
#get number of unique values in Season column
a_data1.SEASON.nunique()

2

In [11]:
#view null/missing values in each column
a_data1.isnull().sum(axis = 0)

MONTH         0
FATALS        0
MonthR        0
SEASON    16441
dtype: int64

In [12]:
#view only columns with null/missing values
[col for col in a_data1.columns if a_data1[col].isnull().any()]

['SEASON']

In [13]:
#  filter out those rows which
# does not contain any data
a_data2 = a_data1.dropna()

In [14]:
#get unique values in source column after recode
a_data2.SEASON.unique()

array(['Winter', 'Fall'], dtype=object)

In [15]:
#view null/missing values in each column
a_data2.isnull().sum(axis = 0)

MONTH     0
FATALS    0
MonthR    0
SEASON    0
dtype: int64

In [16]:
a_data2.describe()


,MONTH,FATALS
count,15725.000000,15725.000000
mean,7.988235,1.089539
std,4.150186,0.365792
min,1.000000,1.000000
25%,2.000000,1.000000
50%,10.000000,1.000000
75%,11.000000,1.000000
max,12.000000,10.000000


In [17]:
def Recode_Season (series):
    if series == 'Winter':
        return 0
    elif series == 'Fall':
        return 1
a_data2['SeasonR'] = a_data1['SEASON'].apply(Recode_Season)
a_data2['SeasonR'].value_counts(sort=False)   

0.0    7117
1.0    8608
Name: SeasonR, dtype: int64

In [18]:
a_data2.head()


,MONTH,FATALS,MonthR,SEASON,SeasonR
0,1,1,January,Winter,0.0
1,1,1,January,Winter,0.0
2,1,1,January,Winter,0.0
3,1,1,January,Winter,0.0
4,1,1,January,Winter,0.0


## Indepent T-test 
### Is there a significant difference in Fatal accidents in Winter months vs the Fall months? 
With the Pvalue of 0.0 there is a significant difference. 

In [21]:
a_data2.groupby(['SEASON'])['FATALS'].describe()

,count,mean,std,min,25%,50%,75%,max
SEASON,,,,,,,,
Fall,8608.0,1.085967,0.350005,1.0,1.0,1.0,1.0,6.0
Winter,7117.0,1.093860,0.384001,1.0,1.0,1.0,1.0,10.0


In [19]:
stats.levene(a_data2['SeasonR'],a_data2['FATALS'])

LeveneResult(statistic=5431.916476149186, pvalue=0.0)

In [22]:
stats.ttest_ind(a_data2['SeasonR'], a_data2['FATALS'], equal_var = False)

Ttest_indResult(statistic=-110.05528083683105, pvalue=0.0)

In [23]:
def welch_dof(x,y):
        dof = (x.var()/x.size + y.var()/y.size)**2 / ((x.var()/x.size)**2 / (x.size-1) + (y.var()/y.size)**2 / (y.size-1))
        print(f"Welch-Satterthwaite Degrees of Freedom= {dof:.4f}")
        
welch_dof(a_data2['SeasonR'], a_data2['FATALS'])

Welch-Satterthwaite Degrees of Freedom= 28872.4494


In [24]:
def welch_ttest(x, y): 
    ## Welch-Satterthwaite Degrees of Freedom ##
    dof = (x.var()/x.size + y.var()/y.size)**2 / ((x.var()/x.size)**2 / (x.size-1) + (y.var()/y.size)**2 / (y.size-1))
   
    t, p = stats.ttest_ind(x, y, equal_var = False)
    
    print("\n",
          f"Welch's t-test= {t:.4f}", "\n",
          f"p-value = {p:.4f}", "\n",
          f"Welch-Satterthwaite Degrees of Freedom= {dof:.4f}")

welch_ttest(a_data2['SeasonR'], a_data2['FATALS'])


 Welch's t-test= -110.0553 
 p-value = 0.0000 
 Welch-Satterthwaite Degrees of Freedom= 28872.4494
